# string_ids, type_ids list, proto_ids list三个重要的list

3个非常重要的表：string,type,proto

* string_ids list
* type_ids list
* proto_ids list

parse method、class时要用到

# 1. parse header

In [1]:
import mmap,struct

f = open('Hello.dex', 'rb')
m = mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ)

string_ids_size = struct.unpack('<L', m[0x38:0x3C])[0] 
string_ids_off  = struct.unpack('<L', m[0x3C:0x40])[0]
type_ids_size   = struct.unpack('<L', m[0x40:0x44])[0]
type_ids_off    = struct.unpack('<L', m[0x44:0x48])[0]
proto_ids_size  = struct.unpack('<L', m[0x48:0x4C])[0]
proto_ids_off   = struct.unpack('<L', m[0x4C:0x50])[0]
field_ids_size  = struct.unpack('<L', m[0x50:0x54])[0]
field_ids_off   = struct.unpack('<L', m[0x54:0x58])[0]

# 2. string_ids list

In [2]:
def string_list():
    string_data = []
    for i in range(string_ids_size):
        off = struct.unpack('<L', m[string_ids_off + (i*4) : string_ids_off + (i*4) + 4 ])[0]
        c_size = m[off]
        c_char = m[off+1:off+1+c_size]
        string_data.append(c_char)
    return string_data
string_list()

[b'<init>',
 b'Hello.java',
 b'LHello;',
 b'Ljava/io/PrintStream;',
 b'Ljava/lang/Object;',
 b'Ljava/lang/String;',
 b'Ljava/lang/System;',
 b'V',
 b'VL',
 b'[Ljava/lang/String;',
 b'hello',
 b'main',
 b'out',
 b'println']

# 3. type_ids list

In [3]:
def typeid_list():
    type_data = []
    for i in range(type_ids_size):
        idx = struct.unpack('<L', m[type_ids_off + (i*4) : type_ids_off + (i*4) + 4])[0]
        type_data.append(idx)
    return type_data
typeid_list()

[2, 3, 4, 5, 6, 7, 9]

对应DexStringId列表中的 [2, 3, 4, 5, 6, 7, 9]，即

```
 b'LHello;',
 b'Ljava/io/PrintStream;',
 b'Ljava/lang/Object;',
 b'Ljava/lang/String;',
 b'Ljava/lang/System;',
 b'V',

 b'[Ljava/lang/String;',
```

这几个就是type（后面parse method和class时会用到）

# 4. proto_ids list

In [5]:
def protoids_list():
    protoids_data = []
    for i in range(proto_ids_size):
        shorty_idx      = struct.unpack('<L', m[proto_ids_off+(i*12)  :proto_ids_off+(i*12)+ 4])[0]# string_ids
        return_type_idx = struct.unpack('<L', m[proto_ids_off+(i*12)+4:proto_ids_off+(i*12)+ 8])[0]# type_ids
        param_off       = struct.unpack('<L', m[proto_ids_off+(i*12)+8:proto_ids_off+(i*12)+12])[0]# address offset
        protoids_data.append([shorty_idx, return_type_idx, param_off])
    return protoids_data
protoids_list()

[[7, 5, 0], [8, 5, 360], [8, 5, 368]]

* shorty_idx ：跟 type_ids 一样 ，它的值是一个 string_ids 的 index 号 ，最终是一个简短的字符串描述 ，用来说明该 method 原型 
* return_type_idx ：它的值是一个 type_ids 的 index 号 ，表示该 method 原型的返回值类型 。
* parameters_off ：后缀 off 是 offset , 指向 method 原型的参数列表 type_list ; 若 method 没有参数 ，值为0 。参数列表的格式是 type_list ，结构从逻辑上如下描述 。size 表示参数的个数 ；type_idx 是对应参数的类型 ，它的值是一个 type_ids 的 index 号 ，跟 return_type_idx 是同一个品种的东西 

 以[7, 5, 0]为例
 
* 7: string_ids list 中index=7，是 'V'
* 5：type_ids list中index=5，是 'V'
* 0：表示没有参数